In [13]:
import os
import json
from pathlib import Path

directory = '30_4/HungKHi_Training/tong'

image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff']
files = os.listdir(directory)
file_dict = {}

for file in files:
    file_path = os.path.join(directory, file)
    
    if os.path.isdir(file_path):
        continue

    file_name, file_ext = os.path.splitext(file)

    if file_name not in file_dict:
        file_dict[file_name] = {"image": None, "json": None}
    
    # Kiểm tra loại file
    if file_ext.lower() in image_extensions:
        file_dict[file_name]["image"] = file
    elif file_ext.lower() == '.json':
        file_dict[file_name]["json"] = file

# Kiểm tra và xóa tệp
files_removed = 0
for base_name, files_info in file_dict.items():
    image_file = files_info["image"]
    json_file = files_info["json"]
    
    # Trường hợp 1: Có ảnh nhưng không có JSON
    if image_file and not json_file:
        image_path = os.path.join(directory, image_file)
        print(f"Xóa ảnh không có JSON tương ứng: {image_path}")
        os.remove(image_path)
        files_removed += 1
    
    # Trường hợp 2: Có JSON nhưng không có ảnh
    elif json_file and not image_file:
        json_path = os.path.join(directory, json_file)
        print(f"Xóa JSON không có ảnh tương ứng: {json_path}")
        os.remove(json_path)
        files_removed += 1
    
    # Trường hợp 3: Có cả ảnh và JSON - kiểm tra JSON có classes không
    elif image_file and json_file:
        json_path = os.path.join(directory, json_file)
        
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
                
            # Kiểm tra nếu JSON không có shapes hoặc không có classes
            if not json_data.get('shapes', []):
                image_path = os.path.join(directory, image_file)
                print(f"Xóa cặp do JSON không có classes: {json_path} và {image_path}")
                os.remove(json_path)
                os.remove(image_path)
                files_removed += 2
        except Exception as e:
            print(f"Lỗi khi đọc file JSON {json_path}: {str(e)}")

print(f"\nĐã xong! Đã xóa {files_removed} tệp không hợp lệ.")
print(f"Còn lại {len(os.listdir(directory))} tệp trong thư mục.")

Xóa ảnh không có JSON tương ứng: 30_4/HungKHi_Training/tong/DJI_0356_W_0226.jpg
Xóa ảnh không có JSON tương ứng: 30_4/HungKHi_Training/tong/DJI_0356_W_0227.jpg
Xóa ảnh không có JSON tương ứng: 30_4/HungKHi_Training/tong/DJI_0356_W_1297.jpg
Xóa ảnh không có JSON tương ứng: 30_4/HungKHi_Training/tong/DJI_0356_W_1298.jpg
Xóa cặp do JSON không có classes: 30_4/HungKHi_Training/tong/9999993_00000_d_0000022.json và 30_4/HungKHi_Training/tong/9999993_00000_d_0000022.jpg
Xóa cặp do JSON không có classes: 30_4/HungKHi_Training/tong/0000220_01546_d_0000005.json và 30_4/HungKHi_Training/tong/0000220_01546_d_0000005.jpg
Xóa cặp do JSON không có classes: 30_4/HungKHi_Training/tong/9999998_00368_d_0000320.json và 30_4/HungKHi_Training/tong/9999998_00368_d_0000320.jpg
Xóa cặp do JSON không có classes: 30_4/HungKHi_Training/tong/0000360_06861_d_0000748.json và 30_4/HungKHi_Training/tong/0000360_06861_d_0000748.jpg
Xóa cặp do JSON không có classes: 30_4/HungKHi_Training/tong/9999937_00000_d_0000041.jso

# Check labels

In [12]:
import os
import json
from collections import Counter

def extract_classes_from_directory(directory_path):
    all_classes = []
    
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                shapes = data.get('shapes', [])
                for shape in shapes:
                    label = shape.get('label')
                    if label:
                        all_classes.append(label)
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
    
    return all_classes

def analyze_classes(classes_list):
    class_counter = Counter(classes_list)
    
    total_annotations = len(classes_list)
    unique_classes = len(class_counter)
    
    print(f"Total annotations: {total_annotations}")
    print(f"Unique classes: {unique_classes}")
    print("\nClass distribution:")
    
    for class_name, count in sorted(class_counter.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total_annotations) * 100
        print(f"{class_name}: {count} ({percentage:.2f}%)")
    
    return class_counter

directory_path = '30_4/HungKHi_Training/tong'
classes = extract_classes_from_directory(directory_path)
class_statistics = analyze_classes(classes)

Total annotations: 158200
Unique classes: 5

Class distribution:
person: 135164 (85.44%)
stick: 8691 (5.49%)
gun2: 7778 (4.92%)
gun1: 6566 (4.15%)
peerson: 1 (0.00%)


# Cắt 1920x1080 -> 1280x7201280x720

In [5]:
import os
import json
import cv2
import numpy as np
import shutil
from pathlib import Path

def process_large_images(input_dir, output_dir, problem_dir=None):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    if problem_dir:
        Path(problem_dir).mkdir(parents=True, exist_ok=True)
    
    target_width, target_height = 1280, 720
    
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for image_file in image_files:
        try:
            image_path = os.path.join(input_dir, image_file)
            image = cv2.imread(image_path)
            
            if image is None:
                print(f"Could not read image: {image_path}")
                continue
            
            height, width = image.shape[:2]
            
            if width <= target_width and height <= target_height:
                print(f"Image {image_file} is already smaller than target size, copying as is.")
                shutil.copy(image_path, os.path.join(output_dir, image_file))
                
                json_file = os.path.splitext(image_file)[0] + '.json'
                json_path = os.path.join(input_dir, json_file)
                if os.path.exists(json_path):
                    # Update JSON file to point to the image file directly
                    with open(json_path, 'r', encoding='utf-8') as f:
                        json_data = json.load(f)
                    
                    # Remove base64 data if it exists and update image path
                    if 'imageData' in json_data:
                        json_data['imageData'] = None
                    if 'imagePath' in json_data:
                        json_data['imagePath'] = image_file
                    
                    output_json_path = os.path.join(output_dir, json_file)
                    with open(output_json_path, 'w', encoding='utf-8') as f:
                        json.dump(json_data, f, indent=4, ensure_ascii=False)
                continue
            
            json_file = os.path.splitext(image_file)[0] + '.json'
            json_path = os.path.join(input_dir, json_file)
            json_data = None
            bboxes = []
            
            if os.path.exists(json_path):
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        json_data = json.load(f)
                    
                    if 'shapes' in json_data:
                        for shape in json_data['shapes']:
                            shape_type = shape.get('shape_type', '')
                            
                            if shape_type == 'polygon' and 'points' in shape and len(shape['points']) >= 3:
                                points = np.array(shape['points'])
                                x_min, y_min = np.min(points, axis=0)
                                x_max, y_max = np.max(points, axis=0)
                                
                                bboxes.append({
                                    'x1': x_min, 'y1': y_min, 'x2': x_max, 'y2': y_max,
                                    'id': shape.get('id', '')
                                })
                            elif (shape_type == 'rectangle' or not shape_type) and 'points' in shape and len(shape['points']) >= 2:
                                points = shape['points']
                                x1, y1 = points[0]
                                x2, y2 = points[1]
                                x1, x2 = min(x1, x2), max(x1, x2)
                                y1, y2 = min(y1, y2), max(y1, y2)
                                
                                bboxes.append({
                                    'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                                    'id': shape.get('id', '')
                                })
                    
                    elif 'objects' in json_data:
                        for obj in json_data['objects']:
                            if 'bbox' in obj:
                                bbox = obj['bbox']
                                if len(bbox) == 4:
                                    if obj.get('format') == 'xyxy':
                                        x1, y1, x2, y2 = bbox
                                    else:
                                        x, y, w, h = bbox
                                        x1, y1, x2, y2 = x, y, x + w, y + h
                                    
                                    bboxes.append({
                                        'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                                        'id': obj.get('id', '')
                                    })
                
                except Exception as e:
                    print(f"Error reading JSON {json_file}: {e}")
                    if problem_dir:
                        shutil.copy(image_path, os.path.join(problem_dir, image_file))
                        if os.path.exists(json_path):
                            shutil.copy(json_path, os.path.join(problem_dir, json_file))
                    continue
            
            start_x, start_y = 0, 0
            
            if bboxes:
                left = min(box['x1'] for box in bboxes)
                right = max(box['x2'] for box in bboxes)
                top = min(box['y1'] for box in bboxes)
                bottom = max(box['y2'] for box in bboxes)
                
                center_x = (left + right) / 2
                center_y = (top + bottom) / 2
                
                start_x = max(0, int(center_x - target_width / 2))
                start_y = max(0, int(center_y - target_height / 2))
                
                if start_x + target_width > width:
                    start_x = max(0, width - target_width)
                if start_y + target_height > height:
                    start_y = max(0, height - target_height)
                
                objects_visible = True
                
                for box in bboxes:
                    box_width = box['x2'] - box['x1']
                    box_height = box['y2'] - box['y1']
                    
                    visible_left = max(box['x1'], start_x)
                    visible_right = min(box['x2'], start_x + target_width)
                    visible_top = max(box['y1'], start_y)
                    visible_bottom = min(box['y2'], start_y + target_height)
                    
                    visible_width = max(0, visible_right - visible_left)
                    visible_height = max(0, visible_bottom - visible_top)
                    
                    width_visibility = visible_width / box_width if box_width > 0 else 0
                    height_visibility = visible_height / box_height if box_height > 0 else 0
                    
                    if width_visibility < 0.7 or height_visibility < 0.7:
                        objects_visible = False
                        break
                
                if not objects_visible and problem_dir:
                    shutil.copy(image_path, os.path.join(problem_dir, image_file))
                    if os.path.exists(json_path):
                        shutil.copy(json_path, os.path.join(problem_dir, json_file))
                    print(f"Objects in {image_file} would be cropped too much, moved to problem directory")
                    continue
            else:
                start_x = max(0, (width - target_width) // 2)
                start_y = max(0, (height - target_height) // 2)
            
            start_x = max(0, min(start_x, width - target_width))
            start_y = max(0, min(start_y, height - target_height))
            
            cropped_image = image[start_y:start_y + target_height, start_x:start_x + target_width]
            
            output_image_path = os.path.join(output_dir, image_file)
            cv2.imwrite(output_image_path, cropped_image)
            
            if json_data and os.path.exists(json_path):
                # Remove imageData to save memory
                if 'imageData' in json_data:
                    json_data['imageData'] = None
                
                if 'shapes' in json_data:
                    for shape in json_data['shapes']:
                        shape_type = shape.get('shape_type', '')
                        
                        if shape_type == 'polygon' and 'points' in shape and len(shape['points']) >= 3:
                            new_points = []
                            for point in shape['points']:
                                x, y = point
                                new_x = max(0, min(target_width, x - start_x))
                                new_y = max(0, min(target_height, y - start_y))
                                new_points.append([new_x, new_y])
                            shape['points'] = new_points
                            
                        elif (shape_type == 'rectangle' or not shape_type) and 'points' in shape and len(shape['points']) >= 2:
                            points = shape['points']
                            x1, y1 = points[0]
                            x2, y2 = points[1]
                            
                            new_x1 = max(0, min(target_width, x1 - start_x))
                            new_y1 = max(0, min(target_height, y1 - start_y))
                            new_x2 = max(0, min(target_width, x2 - start_x))
                            new_y2 = max(0, min(target_height, y2 - start_y))
                            
                            shape['points'] = [[new_x1, new_y1], [new_x2, new_y2]]
                    
                    json_data['imageWidth'] = target_width
                    json_data['imageHeight'] = target_height
                    if 'imagePath' in json_data:
                        json_data['imagePath'] = image_file
                
                elif 'objects' in json_data:
                    for obj in json_data['objects']:
                        if 'bbox' in obj:
                            bbox = obj['bbox']
                            if len(bbox) == 4:
                                if obj.get('format') == 'xyxy':
                                    x1, y1, x2, y2 = bbox
                                    new_x1 = max(0, min(target_width, x1 - start_x))
                                    new_y1 = max(0, min(target_height, y1 - start_y))
                                    new_x2 = max(0, min(target_width, x2 - start_x))
                                    new_y2 = max(0, min(target_height, y2 - start_y))
                                    obj['bbox'] = [new_x1, new_y1, new_x2, new_y2]
                                else:
                                    x, y, w, h = bbox
                                    new_x = max(0, min(target_width, x - start_x))
                                    new_y = max(0, min(target_height, y - start_y))
                                    new_w = min(w, target_width - new_x)
                                    new_h = min(h, target_height - new_y)
                                    obj['bbox'] = [new_x, new_y, new_w, new_h]
                        
                        if 'segmentation' in obj:
                            if isinstance(obj['segmentation'], list):
                                new_segmentation = []
                                for polygon in obj['segmentation']:
                                    new_polygon = []
                                    for i in range(0, len(polygon), 2):
                                        if i + 1 < len(polygon):
                                            x, y = polygon[i], polygon[i+1]
                                            new_x = max(0, min(target_width, x - start_x))
                                            new_y = max(0, min(target_height, y - start_y))
                                            new_polygon.extend([new_x, new_y])
                                    new_segmentation.append(new_polygon)
                                obj['segmentation'] = new_segmentation
                
                if json_data.get('width'):
                    json_data['width'] = target_width
                if json_data.get('height'):
                    json_data['height'] = target_height
                if json_data.get('path'):
                    json_data['path'] = image_file
                
                output_json_path = os.path.join(output_dir, json_file)
                with open(output_json_path, 'w', encoding='utf-8') as f:
                    json.dump(json_data, f, indent=4, ensure_ascii=False)
            
        except Exception as e:
            print(f"Error processing {image_file}: {e}")
            if problem_dir:
                try:
                    shutil.copy(image_path, os.path.join(problem_dir, image_file))
                    json_path = os.path.join(input_dir, os.path.splitext(image_file)[0] + '.json')
                    if os.path.exists(json_path):
                        shutil.copy(json_path, os.path.join(problem_dir, os.path.splitext(image_file)[0] + '.json'))
                except Exception as copy_error:
                    print(f"Error copying to problem directory: {copy_error}")
    
    print(f"Processing completed. Images and annotations saved to {output_dir}")

if __name__ == "__main__":
    input_directory = r"D:\bai_toan\weapon\4_9_min\nhan\tong"
    output_directory = r"D:\bai_toan\weapon\4_9_min\nhan\tong\output"
    problem_directory = r"D:\bai_toan\weapon\4_9_min\nhan\tong\problem"
    
    process_large_images(input_directory, output_directory, problem_directory)

Objects in DJI_0325_1.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_10.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_11.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_12.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_13.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_14.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_15.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_16.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_17.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_18.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_19.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_2.jpg would be cropped too much, moved to problem directory
Objects in DJI_0325_20.jpg wou

In [3]:
import os
from PIL import Image

# Đường dẫn tới thư mục chứa ảnh
folder_path = r"D:\bai_toan\weapon\nen"

# Danh sách phần mở rộng ảnh thường gặp
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']

# Duyệt qua tất cả các file trong thư mục
for filename in os.listdir(folder_path):
    if any(filename.lower().endswith(ext) for ext in image_extensions):
        image_path = os.path.join(folder_path, filename)
        try:
            with Image.open(image_path) as img:
                width, height = img.size
                print(f"{filename}: {width}x{height}")
        except Exception as e:
            print(f"Không thể mở ảnh {filename}: {e}")


1763b32c-946b-4c8c-b385-736f2e8b2c6c.jpeg: 1920x1080
26e5fe34-9fff-4102-a169-b6824f8cb87a.jpeg: 1920x1080
45a8e84c-cf94-46c5-bc9d-f409ed91ffe4.jpeg: 1920x1080
471c0ddb-471a-499e-8910-eb6da8cce176.jpeg: 1920x1080
DJI_0328.JPG: 1920x1080
DJI_0330.JPG: 1920x1080
DJI_0332.JPG: 1920x1080
DJI_0334.JPG: 1920x1080
DJI_0337.JPG: 1920x1080
DJI_0338.JPG: 1920x1080
DJI_0341.JPG: 1920x1080
DJI_0344.JPG: 1920x1080
DJI_0347.JPG: 1920x1080
DJI_0354.JPG: 1920x1080
DJI_0357.JPG: 1920x1080
DJI_0359.JPG: 1920x1080
DJI_0360.JPG: 1920x1080
DJI_0362.JPG: 1920x1080
DJI_0364.JPG: 1920x1080
DJI_0365.JPG: 1920x1080
DJI_0366.JPG: 1920x1080
DJI_0367.JPG: 1920x1080
DJI_0369.JPG: 1920x1080
DJI_0370.JPG: 1920x1080
DJI_0371.JPG: 1920x1080
DJI_0372.JPG: 1920x1080
DJI_0375.JPG: 1920x1080
DJI_0376.JPG: 1920x1080
DJI_0378.JPG: 1920x1080
DJI_0381.JPG: 1920x1080
DJI_0382.JPG: 1920x1080
DJI_0383.JPG: 1920x1080
DJI_0386.JPG: 1920x1080
DJI_0388.JPG: 1920x1080
DJI_0389.JPG: 1920x1080
DJI_0390.JPG: 1920x1080
DJI_0392.JPG: 1920x1

In [ ]:
import cv2
import numpy as np
import json
import random
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

# Đường dẫn tới ảnh và file JSON
image_path = r"D:\bai_toan\weapon\22_4\HungKhi_Tua\test\DJI_0404_W_2496.jpg"
json_path = r"D:\bai_toan\weapon\22_4\HungKhi_Tua\test\DJI_0404_W_2496.json"

# Đọc ảnh
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Chuyển sang RGB để hiển thị với plt

# Đọc file JSON
with open(json_path, 'r') as f:
    label_data = json.load(f)

# Hàm lấy tâm của đa giác (centroid)
def get_centroid(points):
    poly = Polygon(points)
    centroid = poly.centroid
    return np.array([centroid.x, centroid.y])

# Hàm xoay điểm quanh một tâm với góc nhất định
def rotate_point(point, center, angle):
    angle_rad = np.radians(angle)
    x, y = point
    cx, cy = center
    new_x = cx + np.cos(angle_rad) * (x - cx) - np.sin(angle_rad) * (y - cy)
    new_y = cy + np.sin(angle_rad) * (x - cx) + np.cos(angle_rad) * (y - cy)
    return np.array([new_x, new_y])

# Hàm xoay mask của đối tượng
def rotate_mask(mask, center, angle):
    M = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
    rotated_mask = cv2.warpAffine(mask, M, (mask.shape[1], mask.shape[0]), flags=cv2.INTER_NEAREST)
    return rotated_mask

# Hàm lấy vùng bao quanh đối tượng stick
def get_bounding_box(points):
    points = np.array(points)
    x, y = points[:, 0], points[:, 1]
    x_min, x_max = int(np.min(x)), int(np.max(x))
    y_min, y_max = int(np.min(y)), int(np.max(y))
    return x_min, y_min, x_max, y_max

# Tạo ảnh kết quả
result_image = image.copy()

# Xử lý từng đối tượng trong file JSON
for shape in label_data['shapes']:
    if shape['label'] == 'stick':
        # Lấy tọa độ của stick
        stick_points = np.array(shape['points'], dtype=np.float32)
        
        # Tìm đối tượng person để lấy tâm xoay
        person_center = None
        for person_shape in label_data['shapes']:
            if person_shape['label'] == 'person':
                person_points = person_shape['points']
                person_center = get_centroid(person_points)
                break
        
        if person_center is None:
            print("Không tìm thấy đối tượng person!")
            continue

        # Góc xoay ngẫu nhiên từ 10 đến 350 độ
        angle = random.uniform(10, 350)

        # Tạo mask cho stick
        stick_mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.fillPoly(stick_mask, [stick_points.astype(np.int32)], 255)

        # Lấy vùng bao quanh stick
        x_min, y_min, x_max, y_max = get_bounding_box(stick_points)
        x_min = max(0, x_min - 10)  # Mở rộng vùng một chút
        y_min = max(0, y_min - 10)
        x_max = min(image.shape[1], x_max + 10)
        y_max = min(image.shape[0], y_max + 10)

        # Cắt vùng stick từ ảnh gốc
        stick_region = image[y_min:y_max, x_min:x_max].copy()
        stick_region_mask = stick_mask[y_min:y_max, x_min:x_max].copy()

        # Xoay vùng stick
        rotated_mask = rotate_mask(stick_region_mask, (person_center[0] - x_min, person_center[1] - y_min), angle)
        rotated_region = np.zeros_like(stick_region)
        for c in range(3):  # Xoay từng kênh màu
            rotated_region[:, :, c] = cv2.warpAffine(stick_region[:, :, c], cv2.getRotationMatrix2D(
                (person_center[0] - x_min, person_center[1] - y_min), angle, 1.0), 
                (stick_region.shape[1], stick_region.shape[0]))

        # Tạo mask ngược để giữ nguyên phần không phải stick
        inverse_mask = cv2.bitwise_not(rotated_mask)

        # Điền vùng xoay vào ảnh gốc
        temp_region = result_image[y_min:y_max, x_min:x_max].copy()
        temp_region = cv2.bitwise_and(temp_region, temp_region, mask=inverse_mask)
        rotated_region = cv2.bitwise_and(rotated_region, rotated_region, mask=rotated_mask)
        result_image[y_min:y_max, x_min:x_max] = cv2.add(temp_region, rotated_region)

# Hiển thị kết quả
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Ảnh gốc")
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Ảnh sau khi xoay stick")
plt.imshow(result_image)
plt.axis('off')

plt.show()